In [ ]:
#### conda create -n elastic python=3.10
#### conda activate elastic
#### python3 -m pip install sentence-transformers elasticsearch transformers
#### python elasticsearch 8.10.0
#### elasticsearch 8.10.2
#### support knn search ,语义搜索，组合搜索
#### 语言模型转向量


In [5]:
##
from sentence_transformers import SentenceTransformer
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
model


SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [2]:
from elasticsearch import Elasticsearch
client = Elasticsearch(['http://localhost:9200'])
print(client.info())


{'name': 'node-1', 'cluster_name': 'application-es', 'cluster_uuid': 'WbxhejYiSCSbFmNxcAjMwg', 'version': {'number': '8.10.2', 'build_flavor': 'default', 'build_type': 'zip', 'build_hash': '6d20dd8ce62365be9b1aca96427de4622e970e9e', 'build_date': '2023-09-19T08:16:24.564900370Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [23]:
INDEX_NAME = "book_index"

if client.indices.exists(index=INDEX_NAME):
    print("Deleting existing %s" % INDEX_NAME)
    client.options(ignore_status=[400, 404]).indices.delete(index=INDEX_NAME)

	# Define the mapping
request_body = {
	"settings" : {
		"number_of_shards": 2,
		"number_of_replicas": 0
	},
	'mappings': {
		  "properties": {
			 "title_vector": {
				"type": "dense_vector",
				"dims": 384,
				"index": "true",
				"similarity": "cosine"
			  }
		  }
	}
}
# Create the index
client.indices.create(index = INDEX_NAME, settings=request_body["settings"], mappings=request_body["mappings"])

Deleting existing book_index


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'book_index'})

In [25]:

import json
# Load data into a JSON object
with open(r'C:\Users\shuyu-renky\book.json') as f:
	data_json = json.load(f)
	print(data_json)

	actions = []
	for book in data_json:
		actions.append({"index": {"_index": INDEX_NAME}})
		# Transforming the title into an embedding using the model
		book["title_vector"] = model.encode(book["title"]).tolist()
		actions.append(book)
client.bulk(operations=actions)

[{'title': 'The Pragmatic Programmer: Your Journey to Mastery', 'authors': ['andrew hunt', 'david thomas'], 'summary': 'A guide to pragmatic programming for software engineers and developers', 'publish_date': '2019-10-29', 'num_reviews': 30, 'publisher': 'addison-wesley'}, {'title': 'Python Crash Course', 'authors': ['eric matthes'], 'summary': 'A fast-paced, no-nonsense guide to programming in Python', 'publish_date': '2019-05-03', 'num_reviews': 42, 'publisher': 'no starch press'}, {'title': 'Artificial Intelligence: A Modern Approach', 'authors': ['stuart russell', 'peter norvig'], 'summary': 'Comprehensive introduction to the theory and practice of artificial intelligence', 'publish_date': '2020-04-06', 'num_reviews': 39, 'publisher': 'pearson'}, {'title': 'Clean Code: A Handbook of Agile Software Craftsmanship', 'authors': ['robert c. martin'], 'summary': 'A guide to writing code that is easy to read, understand and maintain', 'publish_date': '2008-08-11', 'num_reviews': 55, 'publ

ObjectApiResponse({'errors': False, 'took': 45, 'items': [{'index': {'_index': 'book_index', '_id': 'tLstEosBy5HE9tVwgp4r', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'tbstEosBy5HE9tVwgp4r', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 5, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'trstEosBy5HE9tVwgp4r', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 't7stEosBy5HE9tVwgp4r', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 6, '_primary_term': 1, 'status': 201}}, {'index': {'_index': 'book_index', '_id': 'uLstEosBy5HE9tVwgp4r', '_version': 1, 'result': 'created', '_shards

In [26]:
def pretty_response(response):
	for hit in response['hits']['hits']:
		id = hit['_id']
		publication_date = hit['_source']['publish_date']
		score = hit['_score']
		title = hit['_source']['title']
		summary = hit['_source']['summary']
		publisher = hit["_source"]["publisher"]
		num_reviews = hit["_source"]["num_reviews"]
		authors = hit["_source"]["authors"]
		pretty_output = (f"\nID: {id}\nPublication date: {publication_date}\nTitle: {title}\nSummary: {summary}\nPublisher: {publisher}\nReviews: {num_reviews}\nAuthors: {authors}\nScore: {score}")
		print(pretty_output)

In [27]:
INDEX_NAME = "book_index"
response = client.search(index=INDEX_NAME, knn={
    "field": "title_vector",
    "query_vector": model.encode("Best javascript books?"),
    "k": 10,
    "num_candidates": 100
})

pretty_response(response)


ID: srstEosBy5HE9tVwPJ75
Publication date: 2008-05-15
Title: JavaScript: The Good Parts
Summary: A deep dive into the parts of JavaScript that are essential to writing maintainable code
Publisher: oreilly
Reviews: 51
Authors: ['douglas crockford']
Score: 0.8075248

ID: vLstEosBy5HE9tVwgp4r
Publication date: 2008-05-15
Title: JavaScript: The Good Parts
Summary: A deep dive into the parts of JavaScript that are essential to writing maintainable code
Publisher: oreilly
Reviews: 51
Authors: ['douglas crockford']
Score: 0.8075248

ID: rrstEosBy5HE9tVwPJ75
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Publisher: oreilly
Reviews: 36
Authors: ['kyle simpson']
Score: 0.6946183

ID: uLstEosBy5HE9tVwgp4r
Publication date: 2015-03-27
Title: You Don't Know JS: Up & Going
Summary: Introduction to JavaScript and programming as a whole
Publisher: oreilly
Reviews: 36
Authors: ['kyle simpson']
Score: 0.6946183

ID: ubstE

In [28]:
INDEX_NAME = "book_index"
response = client.search(index=INDEX_NAME, knn={
    "field": "title_vector",
    "query_vector": model.encode("Best javascript books?"),
    "k": 10,
    "num_candidates": 100,
    "filter": {
        "bool": {
                    "should": [
                        {
                         "term": {
                             "publisher.keyword": "addison-wesley"
                         }
                        },
                       {
                        "term": {
                              "authors.keyword": "robert c. martin"
                         }
                       }
                   ],
    
             }
    }
})

pretty_response(response)


ID: qrstEosBy5HE9tVwPJ75
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.61159486

ID: tLstEosBy5HE9tVwgp4r
Publication date: 2019-10-29
Title: The Pragmatic Programmer: Your Journey to Mastery
Summary: A guide to pragmatic programming for software engineers and developers
Publisher: addison-wesley
Reviews: 30
Authors: ['andrew hunt', 'david thomas']
Score: 0.61159486

ID: sbstEosBy5HE9tVwPJ75
Publication date: 2011-05-13
Title: The Clean Coder: A Code of Conduct for Professional Programmers
Summary: A guide to professional conduct in the field of software engineering
Publisher: prentice hall
Reviews: 20
Authors: ['robert c. martin']
Score: 0.57042736

ID: u7stEosBy5HE9tVwgp4r
Publication date: 2011-05-13
Title: The Clean Coder: A Code of Conduct for Professional Programmers
Summary